In [ ]:
%run ../config/init.py

### Creating results folders

In [ ]:
bam_dir = os.path.join(RESULTS, DATASET, 'alignments')
bed_dir = os.path.join(RESULTS, DATASET, 'peak-calling')
result_dir = working_dir(os.path.join(RESULTS, DATASET, 'diffbind'))
sample_table_file = os.path.join(DATA, DATASET, 'sample_table.csv')
sample_table = pandas.read_csv(sample_table_file, keep_default_na=False)
sample_table.head()

### Generating comparison array

In [ ]:
comparisons = []
for s in itertools.combinations(sample_table['condition'].unique(), 2):
    comparisons.append(list(s))
comparisons

### Processing samples

In [ ]:
log_file = 'diffbind.log'

diffbind_yml = {
    'peakcaller': 'narrowpeak',
    'bamDir': {'class': 'Directory', 'path': bam_dir },
    'bedDir': {'class': 'Directory', 'path': bed_dir },
    'factors': [],
    'minMembers': []
}

for c in comparisons:
    minMembers = min(len(sample_table[sample_table['condition'] == c[0]]), len(sample_table[sample_table['condition'] == c[1]]))
    if minMembers >= 2:        
        comp = '{0}_vs_{1}'.format(c[0], c[1])        
        df = sample_table[(sample_table['condition'] == c[0]) | (sample_table['condition'] == c[1])][['sample_name', 'condition', 'replicate']]
        df = df.rename(index=str, columns={'sample_name': 'id', 'condition': 'Condition', 'replicate':'Replicate'})
        df.to_csv(comp + '.tsv', index=None, sep='\t')
        diffbind_yml['factors'].append({'class': 'File', 'path': os.path.join(result_dir, comp + '.tsv')})
        diffbind_yml['minMembers'].append(minMembers)

if diffbind_yml['factors']:
    write_to_yaml(diffbind_yml, 'diffbind.yml')  
    cmd_header = '{} {}/ChIP-Seq/diffbind-parallel.cwl diffbind.yml > {} 2>&1 &'.format(CWLRUNNER, CWLWORKFLOWS, log_file)
    run_command(cmd_header)

### Checking command output
Execute next cell until it prints: **Run completed**

In [ ]:
check_cwl_command_log(log_file)